In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import csv
import os
from tqdm.auto import tqdm
import gc

import scipy
from multiprocessing import cpu_count


import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('qt5agg')

from utils import *         # Local file containing all the functions that we need
import params               # Parameters file. You should tune it for your own experiment

### Cell 1 : Open files

Open all recordings before filtering + sanity check

In [2]:
"""
    Variables
    
    DO NOT CHANGE VALUES HERE UNLESS DEBUG/SPECIFIC USE
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#Link to the actual raw files from the recording listed in the input_file
recording_directory = params.recording_directory

#Loading raw recording files names
recording_names = params.recording_names

# number of triggers samples acquired per second
fs         = params.fs

# Directory where plots will be saved
output_directory = params.output_directory

Nchannels  = params.nb_channels                #256 for standard MEA, 17 for MEA1 Polychrome

"""
    Processing
"""

recording_names = [rec.replace('.raw','') for rec in recording_names]
rec_it = recording_names[:]+['end']
print('Number of recordings: {}\n'.format(len(recording_names)))

#getting onset for next prints
onsets = {}
onsets = recording_onsets(recording_names, path = recording_directory)

#Opening files
print('\nCheck that recordings lengths are consistent with recording names\n') 


for i in range(len(rec_it)-1):    
    print("{} minutes\t--->\t{} : {} -> OK".format(int((onsets[rec_it[i+1]]-onsets[rec_it[i]])/params.fs/60), i, rec_it[i]))


"""Output :

Var :
recordings_names : Ordered list of stimuli names played during experiment
"""   

print('\n\t\t\t------ End Of Cell ------')

Number of recordings: 9


Check that recordings lengths are consistent with recording names

15 minutes	--->	0 : 20240725-meas00-check-30Hz-15px40ch-530nm_1e4 -> OK
30 minutes	--->	1 : 20240725-meas01-check-30Hz-15px40ch-385nm_5e0 -> OK
32 minutes	--->	2 : 20240725-meas02-check-30Hz-15px40ch-385nm_2e1 -> OK
31 minutes	--->	3 : 20240725-meas03-check-30Hz-15px40ch-385nm_5e1 -> OK
64 minutes	--->	4 : 20240725-meas04-2check-15px40ch-v_5e1-y_1e4-30Hz -> OK
60 minutes	--->	5 : 20240725-meas05-2check-15px40ch-v_1e2-y_1e4-30Hz -> OK
40 minutes	--->	6 : 20240725-meas06-2check-15px40ch-v_5e2-y_1e4-30Hz -> OK
8 minutes	--->	7 : 20240725-meas07-MSF-30Hz-530nm_1e4 -> OK
16 minutes	--->	8 : 20240725-meas08-MSF-15Hz-530nm_1e4 -> OK

			------ End Of Cell ------


### Cell 2 : Extract the triggers

#### <center>REQUIRES CELL 1 RUN</center>

Extract triggers from either the visual or holo trigger channel. Automatic detection of Holographic recording. Check that the detection is perform on the right files. Perform triggers sanity checks for visual stimumi. You can plot them later on cell 4. Can take up to more than 1h to run all recordings depending on your experiment length.

In [3]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#name of your experiment for saving the triggers
exp = params.exp

# select MEA (3=2p room) (4=MEA1 Polycrhome)
MEA = params.MEA                       

#the optimal threshhold for detecting stimuli onsets varies with the rig
threshold  = params.threshold           

Nchannels  = params.nb_channels                #256 for standard MEA, 17 for MEA1 Polychrome

# number of triggers samples acquired per second
fs         = params.fs

#The folder in which you want your triggers to be saved 
triggers_directory = params.triggers_directory

#Channel recording triggers in case of holographic stimuli
holo_channel_id   = params.holo_channel_id

#Channel recording triggers in case of visual stimuli
visual_channel_id = params.visual_channel_id 

"""
    Inputs
"""

#you can decide here to extract the triggers only for some recordings. List their indexes here (starting from 0).
select_rec = []    # do only measurement N, put [] or the complet list to call all of them


"""
    Processing
"""

for rec in range(len(recording_names)):
    if select_rec:
        if rec not in select_rec: continue
    
    print('\n-------------   Processing recording {} out of {}   -------------\n'.format(rec+1,len(recording_names)))

    # Creating all files path
    input_file    = os.path.join(recording_directory,recording_names[rec]+'.raw')
    trigger_file  = os.path.join(triggers_directory,'{}_{}_triggers.pkl'.format(exp,recording_names[rec]))
    data_file     = os.path.join(triggers_directory,'{}_{}_triggers_data.pkl'.format(exp,recording_names[rec]))
    
    print('The triggers are extracted from the sorting file:\t{}\nand the results will be saved at:\t\t\t{}'.format(recording_names[rec]+'.raw',trigger_file))
    if os.path.exists(data_file):
        if (str(input('Trigers already extracted previously. Write again files files? Type Y to do so :\n')) != 'Y') : continue
        
    if is_holographic_rec(input_file): 
        #in this case the stimulus was holograpic
        print(r" /!\ HOLOGRAPHIC Recording /!\ ")
        channel_id   = holo_channel_id
        trigger_type = 'holo'
        onsets_file  = os.path.join(triggers_directory,'{}_{}_laser_onsets.npy'.format(exp,recording_names[rec]))
        offsets_file  = os.path.join(triggers_directory,'{}_{}_laser_offsets.npy'.format(exp,recording_names[rec]))
    else: 
        #in this other case the stimulus was visual
        print(r" /!\ VISUAL Recording /!\ ")
        channel_id   = visual_channel_id        
        trigger_type = 'visual'
        
    #Processing of data calling utils functions
    print("Loading Data...")
    channel_id   = visual_channel_id        
    trigger_type = 'visual'      

    data, t_tot    = load_data(input_file, channel_id = channel_id )  #MANUALLY CHANGE HERE IF THE CHANNEL IS 
                                                                     #AUTHOMATICALLY MISDETECTED. IF SO IT SHOULD 
                                                                     #BE BECAUSE OF ALIASING OR BAD TRIGGER QUALITY
    indices        = detect_onsets(data,threshold)
    indices_errors = run_minimal_sanity_check(indices, stim_type = trigger_type)
    
    #Saving data using utils function save_obj
    save_obj({'indices':indices,'duration':t_tot,'trigger_type':trigger_type,'indice_errors':indices_errors}, trigger_file )
    save_obj(data,data_file)
    

    if trigger_type == 'holo':
        save_obj(indices, onsets_file)
    
        offsets = detect_offsets(data)
        save_obj(offsets, offsets_file)    
        
"""
    Output
    
    Saved in triggers_directory :

{experience_name}_{link_file_name}_triggers.pkl (dict) : 
    keys 'indices' --> detected triggers indices, 
         'duration' --> the stimuli duration, 
         'trigger_type' --> the detection visual or holo stimuli, 
         'indice_errors' --> triggers violating sanity check 
         
{experience_name}_{link_file_name}_triggers_data.pkl (numpy array) : raw signal recorded on the trigger channel
"""

print('\n\t\t\t------ End Of Cell ------')


-------------   Processing recording 1 out of 7   -------------

The triggers are extracted from the sorting file:	20240905-meas00-check-30Hz-15px40ch-530nm_1e4-bis.raw
and the results will be saved at:			/home/quetutom/Desktop/20240905/Analysis/trigs/20240925_20240905-meas00-check-30Hz-15px40ch-530nm_1e4-bis_triggers.pkl
 /!\ VISUAL Recording /!\ 
Loading Data...


  0%|          | 0/23364000 [00:00<?, ?it/s]

Minimal sanity checks : Ok on all 34631 triggers

-------------   Processing recording 2 out of 7   -------------

The triggers are extracted from the sorting file:	20240905-meas01-2col_check-30Hz-15px40ch-385nm_5e2-530_1e4.raw
and the results will be saved at:			/home/quetutom/Desktop/20240905/Analysis/trigs/20240925_20240905-meas01-2col_check-30Hz-15px40ch-385nm_5e2-530_1e4_triggers.pkl
 /!\ VISUAL Recording /!\ 
Loading Data...


  0%|          | 0/89292000 [00:00<?, ?it/s]

Minimal sanity checks : Ok on all 131102 triggers

-------------   Processing recording 3 out of 7   -------------

The triggers are extracted from the sorting file:	20240905-meas02-2col_check-30Hz-15px40ch-385nm_1e4-530_1e4.raw
and the results will be saved at:			/home/quetutom/Desktop/20240905/Analysis/trigs/20240925_20240905-meas02-2col_check-30Hz-15px40ch-385nm_1e4-530_1e4_triggers.pkl
 /!\ VISUAL Recording /!\ 
Loading Data...


...

Minimal sanity checks : Ok on all 57002 triggers

-------------   Processing recording 6 out of 7   -------------

The triggers are extracted from the sorting file:	20240905-meas05-check-30Hz-15px40ch-530nm_1e6.raw
and the results will be saved at:			/home/quetutom/Desktop/20240905/Analysis/trigs/20240925_20240905-meas05-check-30Hz-15px40ch-530nm_1e6_triggers.pkl
 /!\ VISUAL Recording /!\ 
Loading Data...


  0%|          | 0/55734000 [00:00<?, ?it/s]

Minimal sanity checks : Ok on all 83103 triggers

-------------   Processing recording 7 out of 7   -------------

The triggers are extracted from the sorting file:	20240905-meas06-check-30Hz-15px40ch-530nm_1e4.raw
and the results will be saved at:			/home/quetutom/Desktop/20240905/Analysis/trigs/20240925_20240905-meas06-check-30Hz-15px40ch-530nm_1e4_triggers.pkl
 /!\ VISUAL Recording /!\ 
Loading Data...


  0%|          | 0/55354000 [00:00<?, ?it/s]

Minimal sanity checks : Ok on all 82502 triggers

			------ End Of Cell ------


### CELL 3 : Plots triggers for sanity check

#### <center>REQUIRES CELL 1 RUN & CELL 2 RUN AT LEAST ONCE FOR THIS EXPERIMENT </center>


Plots the raw trigger signal with the detected triggers and the errors detected. Independently, plots also the detected triggers, should be a perfect diagonal. Third, plots the number of time points gap to the most common trigger duration (ie theoretical_time_per_frame +- ploted value).

#### <center>/!\/!\/!\ Caution on memory leaks /!\/!\/!\ </center> (if you know a solution please let me know)

In [7]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#Experiment name
exp = params.exp

# Optimal threshhold for detecting stimuli onsets varies with the rig
threshold  = params.threshold

# Directory where plots will be saved
output_directory = params.output_directory


"""
    Inputs
"""

#Set True if you want the plots to be saved
save = False

#Define your x-axis ploting window in a tuple (x-min,x-max). Set False to plot the complete data
ploting_range = False


"""
    Ploting
"""

print(*['{} : {}'.format(i,recording_name) for i, recording_name in enumerate(recording_names)], sep="\n")
recordings = [int(rec_id) for rec_id in input("\nSelect recording : ").split()]


plt.close('all')
gc.collect()
plot_idx = 0

for rec in recordings:
    plot_idx+=1
    # Loading data from pickle files created in cell 3
    data    = np.array(load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers_data.pkl'.format(exp,recording_names[rec])))))
    extracted = load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,recording_names[rec]))))
    err = extracted['indice_errors']
    indices = extracted['indices']
    rec_type = extracted["trigger_type"]
    
    # If ploting range is a tuple, reduce the plot to indices between both values of the tuple
    if ploting_range :
        indices = indices[np.logical_and(indices > ploting_range[0], indices < ploting_range[1])]
        data    = data[np.logical_and(np.array(range(len(data))) > ploting_range[0], np.array(range(len(data))) < ploting_range[1])]
        err     = err[np.logical_and(err > ploting_range[0], err < ploting_range[1])]
    
    plt.figure("Trigger sanity check {}".format(plot_idx))
    
    # Top plot with raw trigger signal, threshold of detection, detected triggers and wrong triggers
    plt.subplot(2,1,1)
    plt.title('{}\n{} channel'.format(recording_names[rec],rec_type))
    
    plt.plot(np.linspace(0,len(data)/fs,len(data) ),data)
    plt.plot(indices/fs,data[indices],'.',markersize=2,zorder=10)

    plt.axhline(threshold, color='green')
    plt.scatter(err/fs,data[err], color='red', marker='x',zorder = 15)
    
    # Bottom left plot of triggers indices. Shoule be a perfect diagonal
    plt.subplot(2,2,3)
    plt.plot(indices)
    plt.title('Detected indices')
    
    # Bottom right plot of relative error gap between detect time of frame and mean frame time
    plt.subplot(2,2,4)
    plt.plot(np.diff(np.diff(indices)))
    try :
        plt.title('Duration {} +- error'.format(np.round(np.mean(np.diff(indices)))))
    except :
        plt.title('Duration {} +- error'.format("NOT COMPUTED"))
                  
    plt.tight_layout()
    plt.show(block = True)
    
    # Saving plot if needed
    if save:
        fig_name = os.path.join(output_directory,r'{}_{}.png'.format(recording_names[rec],link_names[rec]))
        plt.savefig(fig_name)


"""
    Output
    
    if save == True
    
    {recording_file_name}_{link_file_name}.png : Plots for a given recording file

"""

print('\n\t\t\t------ End Of Cell ------')

0 : 20240905-meas00-check-30Hz-15px40ch-530nm_1e4-bis
1 : 20240905-meas01-2col_check-30Hz-15px40ch-385nm_5e2-530_1e4
2 : 20240905-meas02-2col_check-30Hz-15px40ch-385nm_1e4-530_1e4
3 : 20240905-meas03-check-30Hz-15px40ch-530nm_1e4
4 : 20240905-meas04-check-30Hz-15px40ch-530nm_1e5
5 : 20240905-meas05-check-30Hz-15px40ch-530nm_1e6
6 : 20240905-meas06-check-30Hz-15px40ch-530nm_1e4



Select recording :  0



			------ End Of Cell ------


### Cell 4 : Test the Refractory Period Violation of neurons 

This cell needs to be run after the manual spike sorting to grade the different cells base on refractory period violation criteria

In [3]:
# Name of your experiment
exp = params.exp

#Path to the folder with the phy output
phy_directory = params.phy_directory

#Extract the good clusters from the spike sorting
cluster_number , good_clusters = extract_cluster_groups(phy_directory)

#Recuperate the RPVs of all good cells
cell_rpvs = get_cell_rpvs(good_clusters,phy_directory) 

#Cluster the different cells in groups
Note_cells=[]

for cell_i in good_clusters :
    violation=cell_rpvs[cell_i]["rpv"]
    if violation<0.01 :
        Note_cells.append([int(cell_i),'A+'])
    elif violation<0.2:
        Note_cells.append([int(cell_i),'A'])
    elif violation<0.5:
        Note_cells.append([int(cell_i),'B'])
    elif violation<0.75:
        Note_cells.append([int(cell_i),'C'])
    else:
        Note_cells.append([int(cell_i),'D'])
        
Note_cells=np.array(Note_cells)

bins=[]
for note in ['A+','A','B','C','D']:
    bins.append(np.count_nonzero(Note_cells==note))
    
fig=plt.figure()
ax=fig.add_subplot(1,1,1)

ax.bar([1,2,3,4,5],bins,align='center')
ax.set_xticks([1,2,3,4,5])
ax.set_xticklabels(['A+','A','B','C','D'])
plt.show()

print(Note_cells)

note_cells_dir=output_directory+'/note_cells'
save_obj(cell_rpvs,note_cells_dir)

Extracting Manually Curated 'Good' clusters
[['1' 'A+']
 ['2' 'A']
 ['3' 'A']
 ['6' 'D']
 ['7' 'B']
 ['11' 'B']
 ['12' 'A']
 ['14' 'A']
 ['19' 'B']
 ['20' 'B']
 ['22' 'D']
 ['26' 'A']
 ['31' 'D']
 ['37' 'A']
 ['39' 'A']
 ['43' 'B']
 ['48' 'A+']
 ['49' 'A']
 ['50' 'D']
 ['52' 'A']
 ['56' 'B']
 ['57' 'A']
 ['58' 'C']
 ['64' 'A']
 ['68' 'D']
 ['70' 'A']
 ['74' 'A']
 ['78' 'D']
 ['81' 'D']
 ['85' 'C']
 ['86' 'A']
 ['88' 'D']
 ['89' 'C']
 ['90' 'D']
 ['91' 'A']
 ['95' 'A']
 ['96' 'C']
 ['101' 'D']
 ['103' 'A']
 ['107' 'C']
 ['118' 'B']
 ['119' 'D']
 ['120' 'C']
 ['121' 'D']
 ['122' 'B']
 ['127' 'A+']
 ['131' 'C']
 ['140' 'D']
 ['146' 'A']
 ['148' 'A']
 ['152' 'A']
 ['153' 'B']
 ['155' 'B']
 ['156' 'D']
 ['157' 'A']
 ['158' 'D']
 ['165' 'D']
 ['167' 'A']
 ['168' 'D']
 ['169' 'A']
 ['171' 'A']
 ['173' 'D']
 ['179' 'A']
 ['180' 'B']
 ['182' 'A+']
 ['183' 'B']
 ['188' 'B']
 ['197' 'D']
 ['202' 'A']
 ['203' 'B']
 ['204' 'D']
 ['208' 'A']
 ['209' 'D']
 ['212' 'D']
 ['214' 'A']
 ['216' 'D']
 ['218

### Cell 5 : Extract data per neurons

#### <center>REQUIRES CELL 1 RUN</center>

Extract all data from phy numpy variables. Create&save a dictionnary containg spikes times in sec for each neuron splited by recording. Depending on your experiment, this can take severeal minutes.

In [4]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

# Name of your experiment
exp = params.exp

#Path to the folder with the phy output
phy_directory = params.phy_directory

#Path to where data should be saved
output_directory = params.output_directory

#Path to rax recording files
recoding_directory = params.recording_directory

#Frequency of sampling of the mea
fs = params.fs


"""
    Processing
"""

rec_onsets    = recording_onsets(recording_names, path = recording_directory)  

# Get cells index and number
cluster_number , good_clusters = extract_cluster_groups(phy_directory)

good_rpv_clus=[]
for clus in good_clusters:
    if cell_rpvs[clus]["rpv"]<0.5:
        good_rpv_clus.append(clus)

print("There are {} good clusters ({} clusters in total)\n".format(len(good_rpv_clus), len(cluster_number)))

# Extract the spike times from the spike sorting files. This can take a few minutes.
print('Spike extraction: ')
all_spike_times = extract_all_spike_times_from_phy(phy_directory)

print('\n')
print('Spike division in recordings per neuron:')
# create a dictionary with another dictionary for each good cluster
good_data = split_spikes_by_recording(all_spike_times, good_rpv_clus, rec_onsets)


# Save the spike data. This can take a few minutes.
good_data_file_name = os.path.join(output_directory,r'{}_fullexp_neurons_data.pkl'.format(exp))
save_obj(good_data,good_data_file_name)

"""
    Output
    
    data (dict) : key 'cluster_id' --> (dict) key 'recording_name' --> This neuron & this recording spikes times in sec

"""

print('\n\t\t\t------ End Of Cell ------')

Extracting Manually Curated 'Good' clusters
There are 108 good clusters (284 clusters in total)

Spike extraction: 


  0%|          | 0/40276836 [00:00<?, ?it/s]



Spike division in recordings per neuron:


  0%|          | 0/108 [00:00<?, ?it/s]


			------ End Of Cell ------
